In [1]:
# from mnist import MNIST

# mnist = MNIST('/home/bnapp/datasets/mnist/')
# images, labels = mnist.load_training()



In [2]:
import numpy as np
import cv2
import misc

import importlib
importlib.reload(misc)

In [3]:
""" Convolutional Neural Network.
Build and train a convolutional neural network with TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
This example is using TensorFlow layers API, see 'convolutional_network_raw' 
example for a raw implementation with variables.
Author: Aymeric Damien - modified by Alexander Rivkind
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf

# Training Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128

# Network Parameters
size=(28,28)
padding_size=(56,56)
num_input = padding_size[0]*padding_size[1] # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
def scale_mnist_mat(image_mat,size=(28,28),orig_size=(28,28),padding=(28,28),offset=(0,0)):
    o=np.zeros([np.shape(image_mat)[0]]+list(padding))
    for ii,image in enumerate(image_mat):
        img_sc=cv2.resize(0.0+np.reshape(image,list(orig_size)), dsize=size)
        #todo: x/y row/column convention
        o[ii,:]=misc.build_mnist_padded([img_sc],y_size=size[1],x_size=size[0],xx=padding[0],yy=padding[1],offset=offset)
    return o    

In [5]:
scaled_train = scale_mnist_mat(mnist.train.images,padding=padding_size)

In [6]:
scaled_test = scale_mnist_mat(mnist.test.images,padding=padding_size)
scaled_test_2x = scale_mnist_mat(mnist.test.images,padding=padding_size,size=(56,56))

In [7]:
scaled_test_1p5x = scale_mnist_mat(mnist.test.images,padding=padding_size,size=(42,42))

In [8]:
# Create the neural network
def mlp_net(x_dict, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('mlp', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]

        # Flatten the data to a 1-D vector for the fully connected layer

        # Fully connected layer (in tf contrib folder for now)
        fc2 = tf.layers.dense(x, 400,activation=tf.nn.relu)
        fc2 = tf.layers.dense(fc2, 200,activation=tf.nn.relu)
        fc2 = tf.layers.dense(fc2, 200,activation=tf.nn.relu)
        fc2 = tf.layers.dense(fc2, 200,activation=tf.nn.relu)
        # Apply Dropout (if is_training is False, dropout is not applied)
#         fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

#         fc2 = tf.layers.dense(fc1,  100,activation=tf.nn.relu)
        # Apply Dropout (if is_training is False, dropout is not applied)
#         fc2 = tf.layers.dropout(fc2, rate=dropout, training=is_training)       

#         fc2 = tf.layers.dense(fc2,  100,activation=tf.nn.relu)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc2 = tf.layers.dropout(fc2, rate=dropout, training=is_training) 
        
        # Output layer, class prediction
        out = tf.layers.dense(fc2, n_classes)

    return out


# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = mlp_net(features, num_classes, dropout, reuse=False,
                            is_training=True)
    logits_test = mlp_net(features, num_classes, dropout, reuse=True,
                           is_training=False)

    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': scaled_train.reshape([-1,padding_size[0] * padding_size[1]])}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': scaled_test.reshape([-1,padding_size[0] * padding_size[1]])}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_eval_distribute': None, '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f49053d0d30>, '_service': None, '_save_summary_steps': 100, '_protocol': None, '_tf_random_seed': None, '_experimental_distribute': None, '_task_type': 'worker', '_device_fn': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_model_dir': '/tmp/tmphqr3e4nj', '_is_chief': True, '_train_distribute': None, '_evaluation_master': ''}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, u

In [9]:
offsets=list(range(14))
errors=[]
for this_offset in offsets:
    scaled_test = scale_mnist_mat(mnist.test.images,padding=padding_size,size=(28,28),offset=(this_offset,this_offset))
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': scaled_test.reshape([-1,padding_size[0]* padding_size[1]])}, y=mnist.test.labels,
        batch_size=batch_size, shuffle=False)
    # Use the Estimator 'evaluate' method
    e = model.evaluate(input_fn)

    print("Testing Accuracy:", e['accuracy'])
    errors.append(e)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-01-09:59:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphqr3e4nj/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-01-01-09:59:50
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9762, global_step = 2000, loss = 0.089661986
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmphqr3e4nj/model.ckpt-2000
Testing Accuracy: 0.9762
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-01-09:59:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphqr3e4nj/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-01-01-

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphqr3e4nj/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-01-01-10:00:08
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.1154, global_step = 2000, loss = 8.317737
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmphqr3e4nj/model.ckpt-2000
Testing Accuracy: 0.1154


In [10]:
e_mlp=[e['accuracy'] for e in errors]

In [11]:
e_mlp

[0.9762,
 0.8982,
 0.5125,
 0.2167,
 0.067,
 0.0367,
 0.0469,
 0.0544,
 0.0635,
 0.0703,
 0.0832,
 0.0984,
 0.1062,
 0.1154]